# Saving and restoring models

In [1]:
# Dataset:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

housing = fetch_california_housing()
X_train_full, X_test, y_train_full, y_test = train_test_split(
    housing.data, housing.target
)
X_train, X_val, y_train, y_val = train_test_split(
    X_train_full, y_train_full
)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

X_train_A, X_train_B = X_train[:, :5], X_train[:, 2:]
X_val_A, X_val_B = X_val[:, :5], X_val[:, 2:]
X_test_A, X_test_B = X_test[:, :5], X_test[:, 2:]
X_new_A, X_new_B = X_test_A[:3], X_test_B[:3]

In [4]:
# Build a model
input_A = keras.layers.Input(shape=[5], name="wide_input")
input_B = keras.layers.Input(shape=[6], name="deep_input")
hidden1 = keras.layers.Dense(30, activation="relu")(input_B)
hidden2 = keras.layers.Dense(30, activation="relu")(hidden1)
concat = keras.layers.concatenate([input_A, hidden2])
output = keras.layers.Dense(1, name="main_output")(concat)
aux_output = keras.layers.Dense(1, name="aux_output")(hidden2) # for regularization
model2 = keras.Model(inputs=[input_A, input_B], outputs=[output, aux_output])

# Each output needs its own loss function - pass a list of loss functions.
model2.compile(loss=["mse", "mse"], loss_weights=[0.9, 0.1], optimizer = "sgd") # the main output gets larger weight.
history = model2.fit(
    [X_train_A, X_train_B], [y_train, y_train], epochs=5,
    validation_data=([X_val_A, X_val_B], [y_val, y_val]))

y_pred_main, y_pred_aux = model2.predict([X_new_A, X_new_B])

Train on 11610 samples, validate on 3870 samples
Epoch 1/5
11610/11610 [==============================] - 1s 125us/sample - loss: 0.9883 - main_output_loss: 0.8947 - aux_output_loss: 1.8288 - val_loss: 0.6047 - val_main_output_loss: 0.5423 - val_aux_output_loss: 1.1662
Epoch 2/5
11610/11610 [==============================] - 1s 61us/sample - loss: 0.5567 - main_output_loss: 0.4979 - aux_output_loss: 1.0859 - val_loss: 0.5407 - val_main_output_loss: 0.4946 - val_aux_output_loss: 0.9558
Epoch 3/5
11610/11610 [==============================] - 1s 60us/sample - loss: 0.5152 - main_output_loss: 0.4749 - aux_output_loss: 0.8764 - val_loss: 0.5021 - val_main_output_loss: 0.4631 - val_aux_output_loss: 0.8528
Epoch 4/5
11610/11610 [==============================] - 1s 64us/sample - loss: 0.5871 - main_output_loss: 0.5644 - aux_output_loss: 0.7923 - val_loss: 0.5111 - val_main_output_loss: 0.4797 - val_aux_output_loss: 0.7937
Epoch 5/5
11610/11610 [==============================] - 1s 63us/sampl

In [6]:
# saving & restoring a model:
# saving:
model2.save("my_keras_model.h5") # saves the model's architecture and the values of all parameters
# It only works for SequentialAPI and FunctionalAPI models -> doesn't work on the subclass models

#loading:
model = keras.models.load_model("my_keras_model.h5")

## Using callbacks
* specify a list of objects that keras will call at the beginning & end of training at each epoch.

In [10]:
checkpoint_cb = keras.callbacks.ModelCheckpoint("my_keras_model.h5", save_best_only=True) # saves only when validation error is the best so far
history = model.fit([X_train_A, X_train_B], [y_train,y_train], epochs=5, validation_data=([X_val_A, X_val_B], [y_val, y_val]), callbacks= [checkpoint_cb]) # saves checkpoints
model = keras.models.load_model("my_keras_model.h5") # roll back to best model

Train on 11610 samples, validate on 3870 samples
Epoch 1/5
11610/11610 [==============================] - 1s 61us/sample - loss: 0.4189 - main_output_loss: 0.4021 - aux_output_loss: 0.5715 - val_loss: 0.4342 - val_main_output_loss: 0.4182 - val_aux_output_loss: 0.5782
Epoch 2/5
11610/11610 [==============================] - 1s 62us/sample - loss: 0.4139 - main_output_loss: 0.3970 - aux_output_loss: 0.5651 - val_loss: 0.4329 - val_main_output_loss: 0.4139 - val_aux_output_loss: 0.6042
Epoch 3/5
11610/11610 [==============================] - 1s 60us/sample - loss: 0.4094 - main_output_loss: 0.3927 - aux_output_loss: 0.5585 - val_loss: 0.4223 - val_main_output_loss: 0.4042 - val_aux_output_loss: 0.5856
Epoch 4/5
11610/11610 [==============================] - 1s 59us/sample - loss: 0.4049 - main_output_loss: 0.3890 - aux_output_loss: 0.5477 - val_loss: 0.4276 - val_main_output_loss: 0.4132 - val_aux_output_loss: 0.5580
Epoch 5/5
11610/11610 [==============================] - 1s 65us/sample

## Early stopping:
* Stop training when there is no further improvement on the val error.

In [11]:
early_stopping_cb = keras.callbacks.EarlyStopping(patience=3, restore_best_weights=True)
# no need to restore best weight when # epochs is large.
history = model.fit([X_train_A, X_train_B], [y_train,y_train], epochs=5, 
                    validation_data=([X_val_A, X_val_B], [y_val, y_val]), 
                    callbacks= [early_stopping_cb]) # saves checkpoints


Train on 11610 samples, validate on 3870 samples
Epoch 1/5
11610/11610 [==============================] - 2s 137us/sample - loss: 0.4056 - main_output_loss: 0.3897 - aux_output_loss: 0.5494 - val_loss: 0.4084 - val_main_output_loss: 0.3906 - val_aux_output_loss: 0.5691
Epoch 2/5
11610/11610 [==============================] - 1s 61us/sample - loss: 0.3997 - main_output_loss: 0.3842 - aux_output_loss: 0.5393 - val_loss: 0.4091 - val_main_output_loss: 0.3922 - val_aux_output_loss: 0.5620
Epoch 3/5
11610/11610 [==============================] - 1s 63us/sample - loss: 0.3969 - main_output_loss: 0.3820 - aux_output_loss: 0.5353 - val_loss: 0.4050 - val_main_output_loss: 0.3885 - val_aux_output_loss: 0.5543
Epoch 4/5
11610/11610 [==============================] - 1s 55us/sample - loss: 0.3913 - main_output_loss: 0.3765 - aux_output_loss: 0.5247 - val_loss: 0.3923 - val_main_output_loss: 0.3772 - val_aux_output_loss: 0.5284
Epoch 5/5
11610/11610 [==============================] - 1s 63us/sampl

In [21]:
# custom early stopping - examine ratio between val_loss and training_loss
class PrintValTrainRatioCallback(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs):
        proint("\n val/train: {:.2f}".format(logs["val_loss"] / logs["loss"]))

# or something like this:
es = keras.callbacks.EarlyStopping(monitor='val_loss', mode="min", verbose=1, patience=3)
history = model.fit([X_train_A, X_train_B], [y_train,y_train], epochs=5, 
                    validation_data=([X_val_A, X_val_B], [y_val, y_val]), 
                    callbacks= [es]) # saves checkpoints


Train on 11610 samples, validate on 3870 samples
Epoch 1/5
11610/11610 [==============================] - 1s 62us/sample - loss: 0.3426 - main_output_loss: 0.3324 - aux_output_loss: 0.4326 - val_loss: 0.3550 - val_main_output_loss: 0.3451 - val_aux_output_loss: 0.4448
Epoch 2/5
11610/11610 [==============================] - 1s 62us/sample - loss: 0.3421 - main_output_loss: 0.3320 - aux_output_loss: 0.4322 - val_loss: 0.3454 - val_main_output_loss: 0.3355 - val_aux_output_loss: 0.4357
Epoch 3/5
11610/11610 [==============================] - 1s 56us/sample - loss: 0.3400 - main_output_loss: 0.3301 - aux_output_loss: 0.4288 - val_loss: 0.3549 - val_main_output_loss: 0.3447 - val_aux_output_loss: 0.4476
Epoch 4/5
11610/11610 [==============================] - 1s 58us/sample - loss: 0.3397 - main_output_loss: 0.3298 - aux_output_loss: 0.4291 - val_loss: 0.3482 - val_main_output_loss: 0.3383 - val_aux_output_loss: 0.4377
Epoch 5/5
11610/11610 [==============================] - 1s 57us/sample